# Project Two Dashboard

## Setup the Jupyter version of Dash

In [43]:
import base64

# Configure OS routines
import os

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import matplotlib.pyplot as plt

# Configure the plotting routines
import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output, State

# Import the AnimalShelter class
from grazioso.crud import AnimalShelter


## Data Manipulation / Model

### Connection details for MongoDB

In [44]:
username = "aacuser"
password = "SNHU1234"
host = "localhost"
port = 27017
db_name = "AAC"
collection_name = "animals"


### Connect to database via CRUD Module

In [45]:
db = AnimalShelter(username, password, host, port, db_name, collection_name)

2025-04-20 09:21:38,004 - grazioso.crud - INFO - Successfully connected to MongoDB at localhost:27017


### Define the rescue type filters based on the specifications

In [46]:
rescue_filters = {
    "all": {
        "animal_type": "Dog"
    }, # No filter - show all animals
    "water": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland",
        ]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
    },
    "mountain": {
        "animal_type": "Dog", 
        "breed": {"$in": [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "disaster": {
        "animal_type": "Dog", 
        "breed": {"$in": [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
}

### Initial data load - all animals

In [47]:
df = pd.DataFrame.from_records(db.read({}))

2025-04-20 09:21:38,057 - grazioso.crud - INFO - Query returned 10000 documents


### For MongoDB v5+, we will receive the `_id` column with an `ObjectID` type. Drop it to avoid issues with the `data_table`

In [48]:
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Dashboard Layout / View

In [49]:
app = Dash(__name__)

### Load the Grazioso Salvare Logo

In [50]:
try:
    image_filename = '../images/grazioso_salvare_logo.png'
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')
except:
    # If the image can't be loaded, we'll use a placeholder
    encoded_image = None

#### Define the dashboard layout

In [51]:
app.layout = html.Div(
    [
        # Header row with logo and title
        html.Div(
            [
                # Logo with URL anchor to SNHU (client requirement)
                html.A(
                    html.Img(
                        src=f"data:image/png;base64,{encoded_image}"
                        if encoded_image
                        else None,
                        style={"height": "100px", "margin-right": "20px"},
                    ),
                    href="https://www.snhu.edu",
                    target="_blank",
                )
                if encoded_image
                else None,
                # Dashboard title
                html.Div(
                    [
                        html.H1("Grazioso Salvare", style={"margin-bottom": "0px"}),
                        html.H3(
                            "Search and Rescue Dog Finder", style={"margin-top": "0px"}
                        ),
                        # Unique identifier with your name as required
                        html.Div(
                            "Dashboard created by Brett Plemons",
                            style={"font-style": "italic"},
                        ),
                    ]
                ),
            ],
            style={
                "display": "flex",
                "align-items": "center",
                "justify-content": "center",
            },
        ),
        html.Hr(),
        # Filter options as radio buttons
        html.Div(
            [
                html.Label("Filter by Rescue Type:"),
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "All Dogs", "value": "all"},
                        {"label": "Water Rescue", "value": "water"},
                        {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
                        {
                            "label": "Disaster Rescue or Individual Tracking",
                            "value": "disaster",
                        },
                    ],
                    value="all",
                    inline=True,
                ),
            ],
            style={"margin": "10px", "text-align": "center"},
        ),
        html.Hr(),
        # Interactive data table
        html.Div(
            [
                dash_table.DataTable(
                    id="datatable-id",
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True}
                        for i in df.columns
                    ],
                    data=df.to_dict("records"),
                    # Interactive features to make it user-friendly
                    editable=False,  # Don't allow editing
                    filter_action="native",  # Allow filtering of data by user
                    sort_action="native",  # Allow sorting of data by user
                    sort_mode="multi",  # Sort across multiple columns
                    column_selectable="single",  # Allow selecting columns
                    row_selectable="single",  # Allow selecting rows
                    row_deletable=False,  # Prevent row deletion
                    selected_columns=[],  # Initially, no columns are selected
                    selected_rows=[],  # Initially, no rows are selected
                    page_action="native",  # All data is passed to the table up-front
                    page_current=0,  # Start on first page
                    page_size=10,  # Show 10 rows per page
                    style_cell={  # Style cells
                        "font-size": "12px",
                        "text-align": "left",
                    },
                    style_data={  # Style data cells
                        "whiteSpace": "normal",
                        "height": "auto",
                    },
                    style_header={  # Style header cells
                        "backgroundColor": "rgb(30, 30, 30)",
                        "color": "white",
                        "fontWeight": "bold",
                    },
                    style_data_conditional=[  # Highlight selected rows
                        {
                            "if": {"row_index": "odd"},
                            "backgroundColor": "rgb(248, 248, 248)",
                        }
                    ],
                )
            ],
            style={"margin": "10px"},
        ),
        html.Br(),
        html.Hr(),
        # Map and chart container - side by side as specified in requirements
        html.Div(
            [
                # Map container
                html.Div(
                    [html.H3("Location of Selected Animal"), html.Div(id="map-id")],
                    style={
                        "width": "49%",
                        "display": "inline-block",
                        "vertical-align": "top",
                    },
                ),
                # Pie chart container
                html.Div(
                    [dcc.Graph(id="pie-chart")],
                    style={"width": "49%", "display": "inline-block", "margin-top": "40px"},
                ),
            ]
        ),
    ]
)

## Interaction Between Components / Controller

#### Callback to update the data table based on the selected filter

In [52]:
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Update the dashboard based on the selected filter type.
    
    Args:
        filter_type: The selected filter type ('all', 'water', 'mountain', 'disaster')
        
    Returns:
        Updated data for the data table
    """
    # Get the filter based on the selected value
    selected_filter = rescue_filters[filter_type]
    
    # Query the database with the selected filter
    filtered_results = db.read(selected_filter)
    
    # Convert results to DataFrame
    df_filtered = pd.DataFrame.from_records(filtered_results)
    
    # Drop the _id column if it exists
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    
    # Return the filtered data
    return df_filtered.to_dict('records')

#### Callback to display the pie chart

In [53]:
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_pie_chart(viewData):
    """
    Update the pie chart based on the filtered data.
    
    Args:
        viewData: The data from the data table
        
    Returns:
        Updated pie chart figure
    """
    if not viewData:
        # Return an empty figure for no data
        return px.pie(names=['No Data'], values=[1], title='No Data Available')
    
    # Convert the data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart showing breed distribution
    if 'breed' in dff.columns and len(dff) > 0:
        # Group by breed and count occurrences
        breed_counts = dff['breed'].value_counts().nlargest(10)  # Top 10 breeds
        
        fig = px.pie(
            names=breed_counts.index,
            values=breed_counts.values,
            title='Top 10 Dog Breeds'
        )
        
        # Update layout for better appearance
        fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
        
        # Return the figure directly
        return fig
    else:
        # Return an empty figure with a message
        return px.pie(names=['No Breed Data'], values=[1], title='No Breed Data Available')

#### Callback to highlight selected cells in the data table

In [54]:

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """
    Highlight selected columns in the data table.
    
    Args:
        selected_columns: List of selected column IDs
        
    Returns:
        Style data for conditional formatting
    """
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#### Callback to update the geolocation map

In [55]:
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    """
    Update the geolocation map based on the selected animal.
    
    Args:
        viewData: The data from the data table
        selected_rows: List of indices of selected rows
        
    Returns:
        Updated map component
    """
    if not viewData:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default to first row if no row is selected
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]
    
    # Check if we have enough rows to display
    if len(dff) <= row:
        # Default to Austin, TX coordinates if no data
        return [
            dl.Map(style={'width': '100%', 'height': '500px'},
                  center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]
    
    # Extract latitude and longitude for the selected animal
    # Default to Austin, TX if the location is missing
    lat = dff.iloc[row]['location_lat'] if 'location_lat' in dff.columns and not pd.isna(dff.iloc[row]['location_lat']) else 30.75
    lon = dff.iloc[row]['location_long'] if 'location_long' in dff.columns and not pd.isna(dff.iloc[row]['location_long']) else -97.48
    
    # Get animal details for tooltip and popup
    breed = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"
    name = dff.iloc[row]['name'] if 'name' in dff.columns and not pd.isna(dff.iloc[row]['name']) else "Unnamed"
    
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tooltip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

## Run the Application

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


2025-04-20 09:21:41,108 - grazioso.crud - INFO - Query returned 5589 documents
